In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# path_prefix = "drive/MyDrive/547 GroupCoding/Final Project/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install gdown -U --no-cache-dir

import gdown

url = 'https://drive.google.com/drive/u/1/folders/1jSW5PpVXPiU6dEP18fmZZ0_b08cS_CHb'

gdown.download_folder(url)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Retrieving folder list


Processing file 1Lv10vWW2bpzhj3nKJou7MvAyTQBcvPIP 20210101-20221026 MISO Actual Energy Price.csv
Processing file 1kY7lZNcMuq_vWOJvAI6edtEVJm4sA5tS 20210101-20221026 MISO Day-Ahead Energy Price.csv
Processing file 1iNmC7b_KTV2oIUOlnHBQs6vUzoN3fMKi 20210101-20221026 MISO Forecasted Load.csv
Processing file 1Qj7N94FI6lkBoEH6nH_KWROed-dtkE1k 20210101-20221026 weather.csv
Processing file 1piBnR0jpb0kB4MgdAP3Dg-H_ur7MWITs 20221001-20221026 MISO Actual Energy Price.csv
Processing file 1Lc1KWJWR-6niPlTdnGs0nSrnkhjLpGaZ 20221001-20221026 MISO Day-Ahead Energy Price.csv
Processing file 11kEqpl8ZMjlJDkX-OLQyepRGsu8eJNod 20221001-20221026 MISO Forecasted Load.csv
Processing file 1G939r3IIQj07i4v20a39GsrvfMflErm9 20221001-20221026 weather.csv
Building directory structure completed


Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=1Lv10vWW2bpzhj3nKJou7MvAyTQBcvPIP
To: /content/Raw data/20210101-20221026 MISO Actual Energy Price.csv
100%|██████████| 3.79M/3.79M [00:00<00:00, 171MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kY7lZNcMuq_vWOJvAI6edtEVJm4sA5tS
To: /content/Raw data/20210101-20221026 MISO Day-Ahead Energy Price.csv
100%|██████████| 5.04M/5.04M [00:00<00:00, 152MB/s]
Downloading...
From: https://drive.google.com/uc?id=1iNmC7b_KTV2oIUOlnHBQs6vUzoN3fMKi
To: /content/Raw data/20210101-20221026 MISO Forecasted Load.csv
100%|██████████| 449k/449k [00:00<00:00, 85.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Qj7N94FI6lkBoEH6nH_KWROed-dtkE1k
To: /content/Raw data/20210101-20221026 weather.csv
100%|██████████| 2.74M/2.74M [00:00<00:00, 134MB/s]
Downloading...
From: https://drive.google.com/uc?id=1piBnR0jpb0kB4MgdAP3Dg-H_ur7MWITs
To: /content/Raw data/20221001-20221026 MIS

['/content/Raw data/20210101-20221026 MISO Actual Energy Price.csv',
 '/content/Raw data/20210101-20221026 MISO Day-Ahead Energy Price.csv',
 '/content/Raw data/20210101-20221026 MISO Forecasted Load.csv',
 '/content/Raw data/20210101-20221026 weather.csv',
 '/content/Raw data/20221001-20221026 MISO Actual Energy Price.csv',
 '/content/Raw data/20221001-20221026 MISO Day-Ahead Energy Price.csv',
 '/content/Raw data/20221001-20221026 MISO Forecasted Load.csv',
 '/content/Raw data/20221001-20221026 weather.csv']

# Read Files

！！！ The raw data is saved in dat/raw data/, if you want to run this jupyter notebook, you need to move those data files out first!

In [7]:
import numpy as np
import pandas as pd

In [8]:
def df_nw_generator (START_DATE, END_DATE):
    # path1 = path_prefix+'dat/%s-%s MISO Actual Energy Price.csv' % (START_DATE, END_DATE)
    # path2 = path_prefix+'dat/%s-%s MISO Day-Ahead Energy Price.csv' % (START_DATE, END_DATE)
    # path3 = path_prefix+'dat/%s-%s MISO Forecasted Load.csv' % (START_DATE, END_DATE)
    path1 = "Raw data/%s-%s MISO Actual Energy Price.csv" % (START_DATE, END_DATE)
    path2 = 'Raw data/%s-%s MISO Day-Ahead Energy Price.csv' % (START_DATE, END_DATE)
    path3 = 'Raw data/%s-%s MISO Forecasted Load.csv' % (START_DATE, END_DATE)
    # read engergy price of this big area
    df1 = pd.read_csv(path1)
    df1 = df1.groupby(["Date","HUB"])["LMP"].sum().unstack()
    df1 = df1.add_prefix("Price.").reset_index().rename_axis([None],axis=1)
    # read Day-ahead energy price
    df2 = pd.read_csv(path2)
    day_ahead = df2[df2["node"]=="ILLINOIS.HUB"].iloc[:,:-1].reset_index()
    # read forecasted load
    df3 = pd.read_csv(path3)
    # integrate
    df = df1.merge(day_ahead, on=["Date"],how="outer")
    df = df.merge(df3,on=["Date"],how="outer")
    df.rename(columns={'lmp': "Price.Day-Ahead", 'Load':'Forecast_load','Date':'Timestamp'},inplace=True)
    # timestamp->Year,Month,Date,Hour
    df["Year"] = pd.DatetimeIndex(df["Timestamp"]).year
    df["Month"] = pd.DatetimeIndex(df["Timestamp"]).month
    df["Day"] = pd.DatetimeIndex(df["Timestamp"]).day
    df["Hour of Day"] = pd.DatetimeIndex(df["Timestamp"]).hour
    df["Day of Week"] = pd.DatetimeIndex(df["Timestamp"]).dayofweek
    # drop useless
    df.drop(columns=["Timestamp","index"],inplace=True)
    return df



In [9]:
def df_w_generator(START_DATE, END_DATE):
    # path = path_prefix+'dat/%s-%s weather.csv' % (START_DATE, END_DATE)
    path = 'Raw data/%s-%s weather.csv' % (START_DATE, END_DATE)
    df = pd.read_csv(path)
    # timestamp->Year,Month,Date,Hour
    df["Year"] = pd.DatetimeIndex(df["datetime"]).year
    df["Month"] = pd.DatetimeIndex(df["datetime"]).month
    df["Day"] = pd.DatetimeIndex(df["datetime"]).day
    df["Hour of Day"] = pd.DatetimeIndex(df["datetime"]).hour
    df["Day of Week"] = pd.DatetimeIndex(df["datetime"]).dayofweek
    # Save as much features as possible for now.
    df.drop(columns=['name','conditions','icon', 'stations','datetime'],inplace=True)
    return df

In [10]:
def df_generator(START_DATE, END_DATE):
    df_nw = df_nw_generator(START_DATE, END_DATE)
    df_w = df_w_generator(START_DATE, END_DATE)
    df = df_w.merge(df_nw, on=["Year","Month","Day","Hour of Day","Day of Week"],how="outer")
    df = df.reindex(columns = ['Year', 'Month', 'Day', "Day of Week", 'Hour of Day','temp', 'feelslike', 'dew', 'humidity', 'precip', 'precipprob',
        'preciptype', 'snow', 'snowdepth', 'windgust', 'windspeed', 'winddir',
        'sealevelpressure', 'cloudcover', 'visibility', 'solarradiation',
        'solarenergy', 'uvindex', 'severerisk', 
        'Price.ARKANSAS.HUB', 'Price.INDIANA.HUB',
        'Price.LOUISIANA.HUB', 'Price.MICHIGAN.HUB', 'Price.MINN.HUB',
        'Price.MS.HUB', 'Price.TEXAS.HUB', 'Price.Day-Ahead', 'Forecast_load', 'Price.ILLINOIS.HUB'])
    return df


In [11]:
start_debug,end_debug = '20221001', '20221026'
start_full,end_full = '20210101', '20221026'
df_debug = df_generator(start_debug,end_debug)
df_full = df_generator(start_full,end_full)

In [ ]:
# Save output
# df_debug.to_csv(path_prefix+'dat/data_debug.csv',index=False)
# df_full.to_csv(path_prefix+'dat/data_full.csv',index=False)

# df_debug.to_pickle(path_prefix+'dat/data_debug.pkl')
# df_full.to_pickle(path_prefix+'dat/data_full.pkl')

In [12]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15937 entries, 0 to 15936
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 15937 non-null  int64  
 1   Month                15937 non-null  int64  
 2   Day                  15937 non-null  int64  
 3   Day of Week          15937 non-null  int64  
 4   Hour of Day          15937 non-null  int64  
 5   temp                 15935 non-null  float64
 6   feelslike            15935 non-null  float64
 7   dew                  15935 non-null  float64
 8   humidity             15935 non-null  float64
 9   precip               15935 non-null  float64
 10  precipprob           15935 non-null  float64
 11  preciptype           1733 non-null   object 
 12  snow                 15935 non-null  float64
 13  snowdepth            15935 non-null  float64
 14  windgust             9268 non-null   float64
 15  windspeed            15935 non-null 

In [13]:
df_debug.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 623
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 624 non-null    int64  
 1   Month                624 non-null    int64  
 2   Day                  624 non-null    int64  
 3   Day of Week          624 non-null    int64  
 4   Hour of Day          624 non-null    int64  
 5   temp                 624 non-null    float64
 6   feelslike            624 non-null    float64
 7   dew                  624 non-null    float64
 8   humidity             624 non-null    float64
 9   precip               624 non-null    float64
 10  precipprob           624 non-null    int64  
 11  preciptype           54 non-null     object 
 12  snow                 624 non-null    int64  
 13  snowdepth            624 non-null    int64  
 14  windgust             624 non-null    float64
 15  windspeed            624 non-null    flo